## **WEEK5 RecSys assignment**

### : **Collaborative Filtering** 직접 구현해보기
<br>

#### 문제는 코드의 빈 부분을 채우는 과제입니다.
<br>

##### 질문 있으시면 22기 조하늘에게 연락주세요

## **Data**

### MovieLens Data
- 이 데이터는 총 3가지의 파일로 구성됩니다.
  > 사용자 데이터 : u.user
  >
  > 영화에 대한 데이터 : u.item
  >
  > 영화 평가에 대한 데이터 : u.data

In [7]:
import os
import pandas as pd

# user 데이터
base_dir = r"C:\Users\njh45\OneDrive\바탕 화면\tobigs_23기\week5\Rec_data" # 경로 재설정하기
u_user_path = os.path.join(base_dir, "u.user")
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(u_user_path,
                    sep='|',
                    names=u_cols,
                    encoding='latin-1'
                    )
users = users.set_index('user_id')
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [8]:
# item 데이터
u_item_path = os.path.join(base_dir, "u.item")
i_cols = ['movie_id', 'title', 'release date', 'video release date',
          'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation',
          'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
          'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
          'Thriller', 'War', 'Western']
movies = pd.read_csv(u_item_path,
                    sep='|',
                    names=i_cols,
                    encoding='latin-1'
                    )
movies = movies.set_index('movie_id')
movies.head()

,title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [9]:
#rating Data
u_data_path = os.path.join(base_dir, "u.data")
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(u_data_path,
                    sep='\t',
                    names=r_cols,
                    encoding='latin-1'
                    )
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [10]:
import numpy as np
from sklearn.model_selection import train_test_split

#RMSE 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# score(RMSE) 계산
def score(model):

    # test 데이터의 user_id와 movie_id 간 pair를 맞춰 튜플형 원소 리스트 데이터를 만듬
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])

    # 모든 사용자-영화 pair 짝에 대해서 주어진 예측 모델에 의해 예측값 계산 및 리스트형 데이터 생성
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])

    y_true = np.array(x_test['rating'])

    return RMSE(y_true, y_pred)


### 데이터셋 만들기 ####
x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

ratings_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')


# 사용자 간 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity

matrix_dummy = ratings_matrix.copy().fillna(0)


user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)

user_similarity = pd.DataFrame(user_similarity,
                                index=ratings_matrix.index,
                                columns=ratings_matrix.index)

## [문제1] Simple CF
### 아래 설명에 따라 직접 구현해볼 것
1. **영화가 평점 매트릭스에 있는지 확인**: 해당 movie_id가 평점 매트릭스에 있는지 확인하여 있으면 다음 단계로 진행.


2. **사용자 유사도와 영화 평점 추출**: user_id의 유사도 점수와 movie_id의 영화 평점을 추출.


3. **평점이 없는 사용자 제거**: 영화에 평점을 매기지 않은 사용자(평점이 없는 사용자)를 제거하고, 그에 해당하는 유사도 점수도 제거.


4. **가중 평균 계산**: 유사도 점수를 가중치로 하여 평점의 가중 평균을 계산해 예측 평점을 생성.


5. **영화가 없을 경우 기본값 사용**: 해당 영화가 매트릭스에 없으면 기본값인 3.0을 예측 평점으로 반환.


6. **결과 반환**: 계산된 예측 평점을 반환.



In [11]:
## 코드 채우기

### 주어진 영화의(movie_id) 가중 평균 rating을 계산하는 함수 ###
def CF_simple(user_id, movie_id):

    # movie_id가 ratings_matrix에 있는 경우에만 아래 코드를 실행
    if movie_id in ratings_matrix.columns:

        # 해당 user_id의 유사도 점수를 복사해옴
        sim_scores = user_similarity[user_id].copy()

        # 해당 movie_id에 대한 영화 평점을 복사해옴
        movie_ratings = ratings_matrix[movie_id].copy()

        # 영화 평점이 없는(null) 인덱스를 찾아 저장
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index

        # null 값이 있는 행을 제거하여 평점만 남김
        movie_ratings.drop(index=none_rating_idx, inplace=True)

        # 유사도 점수에서 null이었던 사용자의 유사도 점수 제거
        sim_scores.drop(index=none_rating_idx, inplace=True)

        # 평점과 유사도 점수를 기반으로 가중 평균을 계산하여 예측 평점을 만듦
        weighted_ratings = movie_ratings * sim_scores
        mean_rating = weighted_ratings.sum() / sim_scores.sum()  # 가중 평균 계산

    else:
        # movie_id가 없으면 기본 평점 3.0을 반환
        mean_rating = 3.0

    # 계산된 평점 반환
    return mean_rating

In [12]:
# 정확도 계산
score(CF_simple)

1.0204669809808065

## [문제2] 이웃을 고려한 CF

In [ ]:
## score 함수
## 유사집단의 크기를 미리 정하기 위해 기존 score 함수에 neighbor_size 인자값 추가

def score(model, neighbor_size=0):

    # test 데이터의 user_id와 movie_id 간 pair를 맞춰 튜플형 원소 리스트 데이터를 만듬
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])

    # 모든 사용자-영화 pair 짝에 대해서 주어진 예측 모델에 의해 예측값 계산 및 리스트형 데이터 생성
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])

    y_true = np.array(x_test['rating'])

    return RMSE(y_true, y_pred)

In [14]:
## 코드 채우기

### Neighbor를 정해서 예측값을 계산하는 함수
def CF_knn(user_id, movie_id, neighbor_size=0):
  if movie_id in ratings_matrix.columns:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = ratings_matrix[movie_id].copy()
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    sim_scores = sim_scores.drop(none_rating_idx)
    # 여기까지는 동일

    # neighbor_size가 0인 경우
    if neighbor_size == 0:

      # 모든 유사도 점수를 사용하여 가중 평균을 계산
      mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()

    # neighbor_size가 0이 아닌 경우
    else:
      if len(sim_scores) > 1:

        ### 유사도 점수가 1보다 큰 경우, 즉 2명 이상의 유사한 사용자가 있을 때

        # neighbor_size가 유사한 사용자 수를 넘지 않도록 제한
        neighbor_size = min(neighbor_size, len(sim_scores))

        # 유사도 점수를 numpy 배열로 변환
        sim_scores = np.array(sim_scores)

        # 영화 평점을 numpy 배열로 변환
        movie_ratings = np.array(movie_ratings)

        # 유사도 점수를 오름차순으로 정렬한 인덱스를 얻음
        user_idx = np.argsort(sim_scores)[::-1]

        # 상위 neighbor_size개의 유사도 점수 선택
        sim_scores = sim_scores[user_idx][:neighbor_size]

        # 상위 neighbor_size개의 평점 선택
        movie_ratings = movie_ratings[user_idx][:neighbor_size]

        # 선택된 유사도와 평점을 사용해 가중 평균 계산
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()

      else:
        # 유사한 사용자가 1명 이하인 경우 기본 평점 3.0을 반환
        mean_rating = 3.0
  else:
    # movie_id가 없거나 neighbor_size가 없을 경우 기본 평점 3.0을 반환
    mean_rating = 3.0

  return mean_rating


In [15]:
#정확도 계산
score(CF_knn, neighbor_size=10)

1.0285914510266663

In [16]:
## CF_knn을 이용하여 사용자에게 영화를 추천하는 기능 구현
## KNN 클러스터를 사용해서 특정 사용자에게 가까운 사용자들을 참고

# train set이 아닌 full matrix로 다시 설정
ratings_matrix_full= ratings.pivot_table(values='rating',
                                     index='user_id',
                                     columns='movie_id')
matrix_dummy = ratings_matrix_full.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity,
                               index=ratings_matrix_full.index,
                               columns=ratings_matrix_full.index)

def recommend_movie(user_id, n_items, neighbor_size = 10):  # 이전에 찾은 최적의 k값을 설정
  user_movie = ratings_matrix_full.loc[user_id].copy()

  for movie in ratings_matrix_full.columns:
    if pd.notnull(user_movie.loc[movie]):
      user_movie.loc[movie] = 0

    else:
      user_movie.loc[movie] = CF_knn(user_id, movie, neighbor_size)

  movie_sort = user_movie.sort_values(ascending=False)[:n_items]
  recom_movies = movies.loc[movie_sort.index]
  recommendations = recom_movies['title']

  return recommendations

- user_id 4번한테 5개를 추천하며 이때, k값은 4번과의 유사도가 가장 높은 상위 k명인 15명으로 설정

In [17]:
recommend_movie(user_id=4, n_items=5, neighbor_size=15)

movie_id
1189                      Prefontaine (1997)
1500               Santa with Muscles (1996)
1467    Saint of Fort Washington, The (1993)
1293                         Star Kid (1997)
1449                  Pather Panchali (1955)
Name: title, dtype: object

## [선택과제] MF(Matrix Factorization)
### 어렵지 않습니다! MF의 원리와 로직을 하나하나 짚어가면서 작성하시면 됩니다 😀

In [ ]:
class MF():
    def __init__(self, ratings, hyper_params, verbose=True):
        # ratings 행렬을 numpy 배열로 변환
        self.R = np.array(ratings)

        # 사용자 수와 아이템 수를 R 행렬의 shape로부터 계산
        self.num_users, self.num_items = np.shape(self.R)

        # 하이퍼파라미터 설정 (잠재 요인 수 K, 학습률 alpha, 정규화 파라미터 beta, 반복 횟수 iterations)
        self.k = hyper_params['K']
        self.alpha = hyper_params['alpha']
        self.beta = hyper_params['beta']
        self.iterations = hyper_params['iterations']

        # 학습 중간 결과를 출력할지 여부
        self.verbose = hyper_params['verbose']


    def rmse(self):
        # R 행렬에서 값이 있는 (0이 아닌) 위치의 좌표를 가져옴
        xs, ys =

        # 예측 값과 실제 값 간의 차이를 저장할 리스트 초기화
        self.predictions = []
        self.errors = []

        # 각 사용자와 아이템의 실제 평점을 예측하고 에러 계산
        for x, y in zip(xs, ys):

            # x번째 사용자, y번째 아이템의 예측 값 계산
            prediction =

            # 예측 값을 리스트에 저장
            self.predictions.append(prediction)

            # 실제 값과 예측 값의 차이를 에러로 저장
            self.errors.append( {채우기} )

        # 예측 값과 에러 리스트를 numpy 배열로 변환
        self.predictions =
        self.errors =

        return np.sqrt(np.mean(self.errors**2))


    def train(self):
        # P 행렬(사용자-잠재 요인 행렬) 초기화 (정규 분포로 난수 생성)
        self.P =

        # Q 행렬(아이템-잠재 요인 행렬) 초기화 (정규 분포로 난수 생성)
        self.Q =

        # 사용자 바이어스(bias) 초기화
        self.b_u =

        # 아이템 바이어스(bias) 초기화
        self.b_d =

        # 전체 평균 평점 계산
        self.b =

        # 값이 있는 R 행렬의 좌표 저장
        rows, columns =

        # 각 사용자와 아이템의 평점 정보를 샘플로 저장
        self.samples = [(i, j, self.R[i, j]) for i, j in zip(rows, columns)]

        # 학습 과정 기록을 위한 리스트
        training_process = []

        # 지정된 반복 횟수만큼 학습 진행
        for i in range(self.iterations):

            # 샘플을 무작위로 섞음
            np.random.shuffle(self.samples)

            # 확률적 경사 하강법(SGD) 수행
            self.sgd()

            # 현재 모델의 RMSE 계산
            rmse = self.rmse()

            # 학습 과정을 기록
            training_process.append((i+1, rmse))

            # verbose가 True인 경우, 10번마다 진행 상황 출력
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: %d ; train RMSE = %.4f" % (i+1, rmse))

        return training_process


    def get_prediction(self, i, j):
        # 사용자 i와 아이템 j에 대한 예측 값 계산 (평균값 + 사용자 바이어스 + 아이템 바이어스 + P와 Q의 내적)
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)

        return prediction


    def sgd(self):
        # 샘플에 대해 확률적 경사 하강법(SGD) 수행
        for i, j, r in self.samples:

            # 사용자 i와 아이템 j에 대한 예측 값 계산
            prediction = self.get_prediction(i, j)

            # 예측 값과 실제 값의 차이(오차) 계산
            e = (r - prediction)

            # 사용자 바이어스 업데이트
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])

            # 아이템 바이어스 업데이트
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

            # P 행렬(사용자-잠재 요인 행렬) 업데이트
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - (self.beta * self.P[i, :]))

            # Q 행렬(아이템-잠재 요인 행렬) 업데이트
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - (self.beta * self.Q[j, :]))

In [ ]:
R_temp = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

hyper_params = {
    'K': 30,
    'alpha': 0.001,
    'beta': 0.02,
    'iterations': 100,
    'verbose': True
}

mf = MF(R_temp, hyper_params)

training_process = mf.train()

Iteration: 10 ; train RMSE = 0.9585
Iteration: 20 ; train RMSE = 0.9374
Iteration: 30 ; train RMSE = 0.9280
Iteration: 40 ; train RMSE = 0.9225
Iteration: 50 ; train RMSE = 0.9182
Iteration: 60 ; train RMSE = 0.9141
Iteration: 70 ; train RMSE = 0.9091
Iteration: 80 ; train RMSE = 0.9022
Iteration: 90 ; train RMSE = 0.8928
Iteration: 100 ; train RMSE = 0.8811
